Bibliotecas

In [19]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [20]:
spark = SparkSession.builder.getOrCreate()

In [21]:
df_titles = spark.read.csv('title_basics.tsv',header=True,sep='\t')
df_ratings = spark.read.csv('title_ratings.tsv',header=True,sep='\t')

In [22]:
df_titles.columns

['tconst',
 'titleType',
 'primaryTitle',
 'originalTitle',
 'isAdult',
 'startYear',
 'endYear',
 'runtimeMinutes',
 'genres']

In [23]:
df_parquet = spark.read.parquet('titles.parquet')

In [24]:
df_parquet.show()

+--------------------+--------------------+
|       originalTitle|              genres|
+--------------------+--------------------+
|          Carmencita|   Documentary,Short|
|Le clown et ses c...|     Animation,Short|
|      Pauvre Pierrot|Animation,Comedy,...|
|         Un bon bock|     Animation,Short|
|    Blacksmith Scene|        Comedy,Short|
|   Chinese Opium Den|               Short|
|Corbett and Court...|         Short,Sport|
|Edison Kinetoscop...|   Documentary,Short|
|          Miss Jerry|       Romance,Short|
|La sortie de l'us...|   Documentary,Short|
|Akrobatisches Pot...|   Documentary,Short|
|L'arrivée d'un tr...|   Documentary,Short|
|Le débarquement d...|   Documentary,Short|
|   L'arroseur arrosé|        Comedy,Short|
| Autour d'une cabine|     Animation,Short|
|Barque sortant du...|   Documentary,Short|
|Italienischer Bau...|   Documentary,Short|
|Das boxende Känguruh|               Short|
|    The Clown Barber|        Comedy,Short|
|      The Derby 1895|Documentar

In [25]:
df_titles.filter(df_titles.genres.contains('Mystery')).select(['originalTitle','genres'])

DataFrame[originalTitle: string, genres: string]

In [26]:
df_join_imdb = df_titles.join(df_ratings, ['tconst'])

In [27]:
df_join_imdb.show(10)

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|           genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+-----------------+-------------+--------+
|tt0000658|    short|The Puppet's Nigh...|Le cauchemar de F...|      0|     1908|     \N|             2|  Animation,Short|          6.5|     212|
|tt0001732|    short|The Lighthouse Ke...|The Lighthouse Ke...|      0|     1911|     \N|            \N|      Drama,Short|          7.1|      15|
|tt0002253|    short|          Home Folks|          Home Folks|      0|     1912|     \N|            17|      Drama,Short|          4.0|      13|
|tt0002473|    short|    The Sands of Dee|    The Sands of Dee|      0|     1912|     \N|            17|    Romance,Short|  

In [ ]:
# Filtro para pegar todos os filmes do Star Wars que foram lançados depois dos anos 2000

In [ ]:
df_join_imdb.where(df_join_imdb.originalTitle.contains('Star Wars')).where('startYear > 2000').show()

In [56]:
# Filtro para pegar todos os filmes acima do rank 8 do imdb

In [70]:
(
    df_join_imdb
        .where('averageRating > 8')
        .write
        .mode('overwrite')
        .parquet('filmes_avaliacao_maior_que_8',)
)

In [73]:
df_maior_que_8 = spark.read.parquet('filmes_avaliacao_maior_que_8/')

In [74]:
df_maior_que_8.show()

+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|   tconst|titleType|        primaryTitle|       originalTitle|isAdult|startYear|endYear|runtimeMinutes|              genres|averageRating|numVotes|
+---------+---------+--------------------+--------------------+-------+---------+-------+--------------+--------------------+-------------+--------+
|tt0060679|    movie|      Matira Manisha|      Matira Manisha|      0|     1966|     \N|           113|               Drama|          9.0|      25|
|tt0301052|    video|The Fantastic Voyage|     Boob Cruise '98|      1|     1998|     \N|            90|                  \N|         10.0|      14|
|tt0319762|    movie|   Looking Your Best|   Looking Your Best|      0|     1989|     \N|            71|                  \N|          9.4|       9|
|tt0326019|    short| Nothing's Permanent|         Prolazi sve|      0|     1994|     \N|             5|  